## UBCF

In [1]:
import numpy as np
import pandas as pd

In [2]:
rating_details = pd.read_csv('ratings_small.csv')
rating_details.head(10)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [3]:
movie_details = pd.read_csv('movies.csv')
movie_details['title'] = movie_details['title'].str.replace(r"\s*\([^()]*\)","")
movie_details.head(10)

d:\users\dell\python\python37\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


,movieId,title,genres
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji,Adventure|Children|Fantasy
2,3,Grumpier Old Men,Comedy|Romance
3,4,Waiting to Exhale,Comedy|Drama|Romance
4,5,Father of the Bride Part II,Comedy
5,6,Heat,Action|Crime|Thriller
6,7,Sabrina,Comedy|Romance
7,8,Tom and Huck,Adventure|Children
8,9,Sudden Death,Action
9,10,GoldenEye,Action|Adventure|Thriller


In [4]:
main_data = pd.merge(rating_details,movie_details,on = 'movieId')
main_data.head(10)

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds,Drama
1,7,31,3.0,851868750,Dangerous Minds,Drama
2,31,31,4.0,1273541953,Dangerous Minds,Drama
3,32,31,4.0,834828440,Dangerous Minds,Drama
4,36,31,3.0,847057202,Dangerous Minds,Drama
5,39,31,3.0,832525157,Dangerous Minds,Drama
6,73,31,3.5,1255591860,Dangerous Minds,Drama
7,88,31,3.0,1239755559,Dangerous Minds,Drama
8,96,31,2.5,1223256331,Dangerous Minds,Drama
9,110,31,4.0,840100695,Dangerous Minds,Drama


In [5]:
avg = main_data
avg.groupby('title')['rating'].mean().sort_values(ascending=False).head(10)

title
Bang, Bang, You're Dead                        5.0
Reality                                        5.0
Mambo Italiano                                 5.0
George Carlin: Jammin' in New York             5.0
Visions of Light: The Art of Cinematography    5.0
George Carlin: It's Bad for Ya!                5.0
Tango                                          5.0
George Carlin: Back in Town                    5.0
Gentlemen of Fortune                           5.0
Victoria                                       5.0
Name: rating, dtype: float64

In [6]:
avg.groupby('title')['rating'].count().sort_values(ascending=False).head()

title
Forrest Gump                          341
Pulp Fiction                          324
Shawshank Redemption, The             311
Silence of the Lambs, The             304
Star Wars: Episode IV - A New Hope    291
Name: rating, dtype: int64

In [7]:
ratings = pd.DataFrame(avg.groupby('title')['rating'].mean())
ratings.head()

,rating
title,
Days of Summer,3.755556
'Hellboy': The Seeds of Creation,2.000000
'Round Midnight,2.250000
'Salem's Lot,3.500000
'Til There Was You,2.625000


In [8]:
ratings['Count of total ratings'] = pd.DataFrame(avg.groupby('title')['rating'].count())
ratings.head()

,rating,Count of total ratings
title,,
Days of Summer,3.755556,45
'Hellboy': The Seeds of Creation,2.000000,1
'Round Midnight,2.250000,2
'Salem's Lot,3.500000,1
'Til There Was You,2.625000,4


In [9]:
from tqdm import tqdm

chunk_size = 50000
chunks = [x for x in range(0, avg.shape[0], chunk_size)]

for i in range(0, len(chunks) - 1):
    print(chunks[i], chunks[i + 1] - 1)

0 49999


In [10]:
pivot_df = pd.DataFrame()

for i in tqdm(range(0, len(chunks) - 1)):
    chunk_df = avg.iloc[ chunks[i]:chunks[i + 1] - 1]
    interactions = (chunk_df.groupby(['userId', 'title'])['rating']
      .sum()
      .unstack()
      .reset_index()
      .fillna(0)
      .set_index('userId')
    )
    pivot_df = pivot_df.append(interactions, sort=False) 

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  5.09it/s]


In [11]:
pivot_df.head()

title,Days of Summer,"'burbs, The",10 Things I Hate About You,101 Dalmatians,12 Angry Men,127 Hours,"13th Warrior, The",2001: A Space Odyssey,28 Days Later,"40-Year-Old Virgin, The",...,Witness,"Wizard of Oz, The",Wolf,"X-Files: Fight the Future, The",X-Men,Yellow Submarine,You've Got Mail,Young Frankenstein,Young Guns,Zero Effect
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
4,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,5.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,...,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0


In [12]:
pivot_df.columns

Index([' Days of Summer', ''burbs, The', '10 Things I Hate About You',
       '101 Dalmatians', '12 Angry Men', '127 Hours', '13th Warrior, The',
       '2001: A Space Odyssey', '28 Days Later', '40-Year-Old Virgin, The',
       ...
       'Witness', 'Wizard of Oz, The', 'Wolf',
       'X-Files: Fight the Future, The', 'X-Men', 'Yellow Submarine',
       'You've Got Mail', 'Young Frankenstein', 'Young Guns', 'Zero Effect'],
      dtype='object', name='title', length=862)

In [13]:
def recommendation(title):
    user_ratings = pivot_df[title]
    similar = pivot_df.corrwith(user_ratings)
    corr =  pd.DataFrame(similar, columns =['Correlation'])
    corr.sort_values('Correlation',ascending=False)
    corr = corr.join(ratings['Count of total ratings'])
    sim_mov = corr[corr['Count of total ratings']>50].sort_values('Correlation',ascending =False)
    rec_mov = sim_mov[1:6]
    rec_mov.reset_index(inplace=True)
    rec_movU = rec_mov.rename(columns = {'title':'What others are watching'})
    rec_movU = rec_movU.drop(columns=['Correlation','Count of total ratings' ]).style.hide_index()
    return rec_movU

In [14]:
recommendation('Pulp Fiction')

What others are watching
Seven
"Silence of the Lambs, The"
"Usual Suspects, The"
Reservoir Dogs
Goodfellas


## CBF

In [15]:
import pandas as pd
from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

pd.set_option('display.max_columns', 100)
df = pd.read_csv('movies_small.csv')
df.head(5)

,Unnamed: 0,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Ratings.Source,Ratings.Value,Metascore,imdbRating,imdbVotes,imdbID,Type,tomatoMeter,tomatoImage,tomatoRating,tomatoReviews,tomatoFresh,tomatoRotten,tomatoConsensus,tomatoUserMeter,tomatoUserRating,tomatoUserReviews,tomatoURL,DVD,BoxOffice,Production,Website,Response
0,1,The Shawshank Redemption,1994,R,14 Oct 1994,142 min,"Crime, Drama",Frank Darabont,"Stephen King (short story ""Rita Hayworth and S...","Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...,English,USA,Nominated for 7 Oscars. Another 19 wins & 30 n...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.3/10,80.0,9.3,"1,825,626",tt0111161,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/shawshank_rede...,27 Jan 1998,NaN,Columbia Pictures,NaN,True
1,2,The Godfather,1972,R,24 Mar 1972,175 min,"Crime, Drama",Francis Ford Coppola,"Mario Puzo (screenplay), Francis Ford Coppola ...","Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...,"English, Italian, Latin",USA,Won 3 Oscars. Another 23 wins & 27 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.2/10,100.0,9.2,"1,243,444",tt0068646,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather/,09 Oct 2001,NaN,Paramount Pictures,http://www.thegodfather.com,True
2,3,The Godfather: Part II,1974,R,20 Dec 1974,202 min,"Crime, Drama",Francis Ford Coppola,"Francis Ford Coppola (screenplay), Mario Puzo ...","Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...,"English, Italian, Spanish, Latin, Sicilian",USA,Won 6 Oscars. Another 10 wins & 20 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,85.0,9.0,"856,870",tt0071562,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather_part...,24 May 2005,NaN,Paramount Pictures,http://www.thegodfather.com/,True
3,4,The Dark Knight,2008,PG-13,18 Jul 2008,152 min,"Action, Crime, Drama",Christopher Nolan,"Jonathan Nolan (screenplay), Christopher Nolan...","Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...,"English, Mandarin","USA, UK",Won 2 Oscars. Another 151 wins & 153 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,82.0,9.0,"1,802,351",tt0468569,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/the_dark_knight/,09 Dec 2008,"$533,316,061",Warner Bros. Pictures/Legendary,http://thedarkknight.warnerbros.com/,True
4,5,12 Angry Men,1957,APPROVED,01 Apr 1957,96 min,"Crime, Drama",Sidney Lumet,"Reginald Rose (story), Reginald Rose (screenplay)","Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...,English,USA,Nominated for 3 Oscars. Another 16 wins & 8 no...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,8.9/10,96.0,8.9,"494,215",tt0050083,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/1000013-12_ang...,06 Mar 2001,NaN,Criterion Collection,http://www.criterion.com/films/27871-12-angry-men,True


In [16]:
df = df[['Title','Genre','Director','Actors','Plot']]
df.head()


,Title,Genre,Director,Actors,Plot
0,The Shawshank Redemption,"Crime, Drama",Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...
1,The Godfather,"Crime, Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...
2,The Godfather: Part II,"Crime, Drama",Francis Ford Coppola,"Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...
3,The Dark Knight,"Action, Crime, Drama",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...
4,12 Angry Men,"Crime, Drama",Sidney Lumet,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...


In [17]:
# discarding the commas between the actors' full names and getting only the first three names
df['Actors'] = df['Actors'].map(lambda x: x.split(',')[:3])

# putting the genres in a list of words
df['Genre'] = df['Genre'].map(lambda x: x.lower().split(','))

df['Director'] = df['Director'].map(lambda x: x.split(' '))

# merging together first and last name for each actor and director, so it's considered as one word 
# and there is no mix up between people sharing a first name
for index, row in df.iterrows():
    row['Actors'] = [x.lower().replace(' ','') for x in row['Actors']]
    row['Director'] = ''.join(row['Director']).lower()

In [18]:
# initializing the new column
df['Key_words'] = ""

for index, row in df.iterrows():
    plot = row['Plot']
    
    # instantiating Rake, by default is uses english stopwords from NLTK
    # and discard all puntuation characters
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(plot)

    # getting the dictionary whith key words and their scores
    key_words_dict_scores = r.get_word_degrees()
    
    # assigning the key words to the new column
    row['Key_words'] = list(key_words_dict_scores.keys())

# dropping the Plot column
df.drop(columns = ['Plot'], inplace = True)

In [19]:
df.set_index('Title', inplace = True)
df.head()

,Genre,Director,Actors,Key_words
Title,,,,
The Shawshank Redemption,"[crime, drama]",frankdarabont,"[timrobbins, morganfreeman, bobgunton]","[common, decency, finding, solace, eventual, r..."
The Godfather,"[crime, drama]",francisfordcoppola,"[marlonbrando, alpacino, jamescaan]","[reluctant, son, organized, crime, dynasty, tr..."
The Godfather: Part II,"[crime, drama]",francisfordcoppola,"[alpacino, robertduvall, dianekeaton]","[tightens, grip, expands, family, crime, syndi..."
The Dark Knight,"[action, crime, drama]",christophernolan,"[christianbale, heathledger, aaroneckhart]","[gotham, joker, emerges, chaos, fight, injusti..."
12 Angry Men,"[crime, drama]",sidneylumet,"[martinbalsam, johnfiedler, leej.cobb]","[miscarriage, evidence, reconsider, justice, c..."


In [20]:
df['bag_of_words'] = ''
columns = df.columns
for index, row in df.iterrows():
    words = ''
    for col in columns:
        if col != 'Director':
            words = words + ' '.join(row[col])+ ' '
        else:
            words = words + row[col]+ ' '
    row['bag_of_words'] = words
    
df.drop(columns = [col for col in df.columns if col!= 'bag_of_words'], inplace = True)

In [21]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(df['bag_of_words'])

# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use later to match the indexes
indices = pd.Series(df.index)
indices[:5]

0    The Shawshank Redemption
1               The Godfather
2      The Godfather: Part II
3             The Dark Knight
4                12 Angry Men
Name: Title, dtype: object

In [22]:
# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)
pd.DataFrame(cosine_sim)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,...,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249
0,1.000000,0.157895,0.137649,0.083771,0.111283,0.046829,0.095673,0.244558,0.044992,0.046829,0.051299,0.000000,0.000000,0.045883,0.052632,0.170406,0.00000,0.039936,0.210526,0.051299,0.097823,0.091766,0.089984,0.050063,0.046829,0.055641,0.108148,0.052632,0.054074,0.143509,0.000000,0.000000,0.054074,0.000000,0.108148,0.045883,0.000000,0.040555,0.051299,0.046829,0.105263,0.046829,0.000000,0.050063,0.050063,0.000000,0.000000,0.054074,0.052632,0.046829,...,0.108148,0.041204,0.000000,0.000000,0.000000,0.111283,0.051299,0.000000,0.091766,0.086711,0.052632,0.048912,0.146735,0.052632,0.102598,0.047836,0.055641,0.047836,0.118470,0.000000,0.052632,0.043355,0.047836,0.054074,0.045883,0.000000,0.220755,0.048912,0.054074,0.044992,0.000000,0.061314,0.059235,0.047836,0.057354,0.057354,0.052632,0.050063,0.000000,0.059235,0.000000,0.000000,0.000000,0.057354,0.050063,0.057354,0.048912,0.052632,0.052632,0.055641
1,0.157895,1.000000,0.367065,0.125656,0.166924,0.046829,0.047836,0.146735,0.044992,0.046829,0.051299,0.041885,0.000000,0.045883,0.052632,0.213007,0.00000,0.039936,0.157895,0.051299,0.146735,0.137649,0.134976,0.050063,0.046829,0.055641,0.162221,0.052632,0.054074,0.143509,0.000000,0.000000,0.054074,0.000000,0.162221,0.045883,0.047836,0.000000,0.102598,0.046829,0.052632,0.140488,0.000000,0.100125,0.050063,0.000000,0.000000,0.054074,0.052632,0.093659,...,0.162221,0.041204,0.000000,0.000000,0.000000,0.055641,0.051299,0.000000,0.045883,0.130066,0.052632,0.048912,0.146735,0.105263,0.153897,0.095673,0.055641,0.000000,0.177705,0.000000,0.052632,0.043355,0.047836,0.000000,0.045883,0.000000,0.220755,0.097823,0.054074,0.044992,0.000000,0.061314,0.059235,0.047836,0.057354,0.057354,0.052632,0.050063,0.000000,0.059235,0.000000,0.051299,0.000000,0.114708,0.050063,0.057354,0.048912,0.052632,0.052632,0.055641
2,0.137649,0.367065,1.000000,0.109545,0.145521,0.040825,0.041703,0.127920,0.078446,0.040825,0.044721,0.000000,0.000000,0.080000,0.045883,0.259973,0.00000,0.000000,0.137649,0.134164,0.127920,0.120000,0.156893,0.043644,0.040825,0.048507,0.141421,0.091766,0.047140,0.125109,0.000000,0.000000,0.047140,0.000000,0.141421,0.040000,0.041703,0.000000,0.134164,0.040825,0.045883,0.122474,0.000000,0.043644,0.043644,0.036515,0.000000,0.047140,0.045883,0.122474,...,0.141421,0.071842,0.000000,0.043644,0.000000,0.048507,0.044721,0.000000,0.040000,0.113389,0.045883,0.042640,0.127920,0.091766,0.178885,0.083406,0.048507,0.000000,0.154919,0.000000,0.045883,0.075593,0.041703,0.000000,0.040000,0.000000,0.230940,0.127920,0.047140,0.039223,0.041703,0.053452,0.103280,0.125109,0.100000,0.050000,0.045883,0.043644,0.044721,0.051640,0.000000,0.000000,0.000000,0.150000,0.087287,0.100000,0.042640,0.045883,0.045883,0.048507
3,0.083771,0.125656,0.109545,1.000000,0.088561,0.037268,0.076139,0.077850,0.071611,0.111803,0.040825,0.066667,0.074536,0.036515,0.041885,0.101710,0.08165,0.063564,0.083771,0.040825,0.116775,0.073030,0.071611,0.079682,0.037268,0.088561,0.086066,0.041885,0.043033,0.152277,0.000000,0.045644,0.043033,0.043033,0.086066,0.036515,0.114208,0.032275,0.081650,0.037268,0.167542,0.037268,0.043033,0.039841,0.039841,0.066667,0.000000,0.086066,0.041885,0.074536,...,0.086066,0.098374,0.037268,0.119523,0.038925,0.044281,0.040825,0.039841,0.036515,0.069007,0.041885,0.038925,0.077850,0.041885,0.081650,0.114208,0.044281,0.000000,0.094281,0.114208,0.041885,0.034503,0.076139,0.043033,0.073030,0.000000,0.105409,0.038925,0.086066,0.035806,0.000000,0.048795,0.047140,0.038069,0.045644,0.045644,0.041885,0.039841,0.000000,0.047140,0.038925,0.081650,0.000000,0.045644,0.039841,0.045644,0.038925,0.041885,0.041885,0.044281
4,0.111283,0.166924,0.145521,0.0885

In [23]:
# function that takes in movie title as input and returns the top 5 recommended movies
def recommendations(title, cosine_sim = cosine_sim):
    
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 5 most similar movies
    top_5_indexes = list(score_series.iloc[1:6].index)
    
    # populating the list with the titles of the best 5 matching movies
    for i in top_5_indexes:
        recommended_movies.append(list(df.index)[i])
    
    rec_movC = pd.DataFrame (recommended_movies,columns=['Recommended based on Content']).style.hide_index()
   
    return rec_movC

In [24]:
recommendations('Pulp Fiction')

Recommended based on Content
The Shawshank Redemption
Rope
Goodfellas
Reservoir Dogs
Casino


In [25]:
def hybrid(title, cosine_sim = cosine_sim):
    #CBF
    recommended_movies = []
    idx = indices[indices == title].index[0]
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    top_5_indexes = list(score_series.iloc[1:6].index)
    for i in top_5_indexes:
        recommended_movies.append(list(df.index)[i])
    
    rec_movC = pd.DataFrame (recommended_movies,columns=['Recommended based on Content'])
   
    #UBCF
    user_ratings = pivot_df[title]
    similar = pivot_df.corrwith(user_ratings)
    corr =  pd.DataFrame(similar, columns =['Correlation'])
    corr.sort_values('Correlation',ascending=False)
    corr = corr.join(ratings['Count of total ratings'])
    sim_mov = corr[corr['Count of total ratings']>50].sort_values('Correlation',ascending =False)
    rec_mov = sim_mov[1:6]
    rec_mov.reset_index(inplace=True)
    rec_movU = rec_mov.rename(columns = {'title':'What others are watching'})
    rec_movU = rec_movU.drop(columns=['Correlation','Count of total ratings' ])
    
    hyb = pd.concat([rec_movC, rec_movU], axis=1).style.hide_index()
    return hyb
    

In [26]:
hybrid('Reservoir Dogs')

Recommended based on Content,What others are watching
Fargo,Goodfellas
The Departed,"Big Lebowski, The"
Pulp Fiction,Jackie Brown
The Usual Suspects,Full Metal Jacket
No Country for Old Men,"Clockwork Orange, A"


In [27]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import pickle

In [28]:
filename = 'nlp_model.pkl'
clf = pickle.load(open(filename, 'rb'))
vectorizer = pickle.load(open('tranform.pkl','rb'))

### Testing for movie from dataset

In [29]:
dff = pd.read_csv("movies_small.csv")

In [30]:
def search(title):
    filtered = dff[dff["Title"] == title]["imdbID"]
    imdbID = pd.DataFrame(filtered).to_string(columns=None, col_space=None, header=False, index=False,)
    url1 = 'https://www.imdb.com/title/'+imdbID+'/reviews?sort=helpfulnessScore&dir=desc&ratingFilter=0'
    x = requests.get(url1)
    soup = BeautifulSoup(x.text,"html.parser")
    mov_cont = soup.find_all('div', class_ = 'text show-more__control')
    reviews_list = [] # list of reviews
    reviews_status = [] # list of comments (good or bad)
    for reviews in mov_cont:
        if reviews.string:
            reviews_list.append(reviews.string)
            movie_review_list = np.array([reviews.string])
            movie_vector = vectorizer.transform(movie_review_list)
            pred = clf.predict(movie_vector)
            reviews_status.append('Good' if pred else 'Bad')
    scrap = pd.DataFrame({'Review':reviews_list[:8],'Sentiment':reviews_status[:8]})
    scrap.loc[:, 'Review'] = scrap['Review'].str.replace("\n","")
    return scrap

In [31]:
search('Pulp Fiction')

,Review,Sentiment
0,"This is Tarantino's masterpiece, there's no ot...",Good
1,Just the best movie... I can imagine my family...,Bad
2,"Pulp Fiction is the most original, rule breaki...",Bad
3,Before I saw this I assumed it was probably ov...,Good
4,One of the best movies I have ever seen is a f...,Good
5,Can't help but love it!🖤 bcoz it's good cool b...,Bad
6,A pure tarantino movie and the best film for t...,Good
7,"To put this in context, I am 34 years old and ...",Good


In [32]:
def complete(title):
    return display(hybrid(title),search(title).style.hide_index())

In [33]:
complete('Pulp Fiction')

Recommended based on Content,What others are watching
The Shawshank Redemption,Seven
Rope,"Silence of the Lambs, The"
Goodfellas,"Usual Suspects, The"
Reservoir Dogs,Reservoir Dogs
Casino,Goodfellas


Review,Sentiment
"This is Tarantino's masterpiece, there's no other way to say it. It has arguably one of the smartest scripts I've ever seen. The story, which is non-linear, is so well constructed it takes several viewings to grasp it all. The movie doesn't seem to be about any spesific thing, but there is a subtle hint of redemption as a central theme. The characters and preformances in this movie are practically perfect. This is still one of the best performances I've seen from Sam Jackson, and it's an outrage he didn't win an Oscar. Each scene has its own unique flavour and charm, every segment has its own arc while also tying into the main plot. The comedy is great, the serious moments are great, every word of dialogue is exciting despite seemingly not having any reason to exist. This movie is just such a great time, and I recommend it to everyone who loves movies. I cannot think of a single genuine flaw with it, and it will remain one of my favorite movies for a long time.",Good
Just the best movie... I can imagine my family seeing this movie in 30 years. I really love this movie and his soundtrack.,Bad
"Pulp Fiction is the most original, rule breaking film I have ever seen. Instead of following the widely used 3 act structure, Pulp Fiction makes up its own and while the 3 stories may seem completely disconnected at first, once you look closely you can find the underlying themes that they all share. Anyone who says that the movie lacks focus or has no meaning hasn't analysed enough. I highly recommend this film since it is number one on my list of my favourite movies of all time.",Bad
Before I saw this I assumed it was probably overrated. I was wrong. It lives up to and surpasses its reputation in pretty much every way. It has probably changed the way I will look at movies for the rest of my life.,Good
"One of the best movies I have ever seen is a fun and enjoyable story, as most of the film's characters were excellent, and it brought back the character of Butch and his girlfriend, Vincent was the best character and his death was shocking and I did not. Expect that, the interconnectedness of the film's stories was excellent, the direction in the film was excellent The film, the acting in the movie was very cool, I recommend it, and it is the best film by Quentin Tarantino",Good
Can't help but love it!🖤 bcoz it's good cool brilliant? I watched it over agian and again.,Bad
A pure tarantino movie and the best film for tarantino,Good
"To put this in context, I am 34 years old and I have to say that this is the best film I have seen without doubt and I don't expect it will be beaten as far as I am concerned. Obviously times move on, and I acknowledge that due to its violence and one particularly uncomfortable scene this film is not for everyone, but I still remember watching it for the first time, and it blew me away. Anyone who watches it now has to remember that it actually changed the history of cinema. In context- it followed a decade or more of action films that always ended with a chase sequence where the hero saved the day - you could have written those films yourself. Pulp had you gripped and credited the audience with intelligence. There is not a line of wasted dialogue and the movie incorporates a number of complexities that are not immediately obvious. It also resurrected the career of Grease icon John Travolta and highlighted the acting talent of Samuel L Jackson. There are many films now that are edited out of sequence and have multiple plots etc but this is the one they all want to be, or all want to beat, but never will.",Good
